In [1]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
import os



In [2]:
#loading data
DATA_PATH = 'D:\Thesis V2X\Project Implimentation\Berlin V2X\Implementation\cellular_dataframe.parquet'


df = pd.read_parquet(DATA_PATH)


    

In [3]:
df.reset_index(drop=False, inplace=True)

In [4]:
df.head()

,timestamp,device,ping_ms,datarate,jitter,ts_gps,Latitude,Longitude,Altitude,speed_kmh,...,SCell_MNC_Digit,SCell_MNC,SCell_Allowed_Access,SCell_freq_MHz,scenario,drive_mode,target_datarate,direction,measured_qos,operator
0,2021-06-22 09:49:00+02:00,pc4,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2.0,1.0,0.0,2600.0,A3D,platoon,350000000,downlink,datarate,1
1,2021-06-22 09:49:00+02:00,pc3,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2.0,2.0,0.0,1800.0,A3D,platoon,350000000,downlink,datarate,2
2,2021-06-22 09:49:00+02:00,pc1,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2.0,1.0,0.0,2600.0,A3D,platoon,350000000,downlink,datarate,1
3,2021-06-22 09:49:00+02:00,pc2,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2.0,2.0,0.0,2600.0,A3D,platoon,350000000,downlink,datarate,2
4,2021-06-22 09:49:01+02:00,pc2,NaN,NaN,NaN,NaT,NaN,NaN,NaN,NaN,...,2.0,2.0,0.0,2600.0,A3D,platoon,350000000,downlink,datarate,2


In [5]:
corr = df.select_dtypes(include="number").corr()

upper = corr.where(np.triu(np.ones(corr.shape), k=1).astype(bool))

all_pairs = (
    upper.stack()
         .reset_index()
         .rename(columns={"level_0": "col_1", "level_1": "col_2", 0: "corr"})
         .sort_values(by="corr", ascending=False)
)

all_pairs


,col_1,col_2,corr
7414,PCell_Downlink_frequency,PCell_Uplink_frequency,1.000000
9160,SCell_Downlink_RBs_MCS_17,SCell_Downlink_RBs_MCS_27,1.000000
1332,temperature,apparentTemperature,1.000000
8656,SCell_Downlink_RBs_MCS_2,SCell_Downlink_RBs_MCS_17,1.000000
9161,SCell_Downlink_RBs_MCS_17,SCell_Downlink_RBs_MCS_28,1.000000
...,...,...,...
7432,PCell_Downlink_frequency,SCell_E-ARFCN,-0.965190
7496,PCell_Uplink_frequency,SCell_E-ARFCN,-0.965190
7847,PCell_freq_MHz,SCell_freq_MHz,-0.990452
7726,PCell_Band_Indicator,SCell_freq_MHz,-0.998626


In [6]:
all_pairs.assign(abs_corr=all_pairs["corr"].abs()).sort_values("abs_corr", ascending=False)
corr_pairs = all_pairs.to_csv('results/corr_pairs.csv')


In [7]:
# 1) Missing % per feature
missing_pct = (
    df.isna()
    .mean()
    .mul(100)
    .round(2)
)

# 2) Sort features by missing %
missing_pct_sorted = missing_pct.sort_values(ascending=True)

# 3) Split sorted features into requested ranges
sorted_missing_ranges = {
    "0-10%": missing_pct_sorted[(missing_pct_sorted >= 0) & (missing_pct_sorted < 10)],
    "10-70%": missing_pct_sorted[(missing_pct_sorted >= 10) & (missing_pct_sorted < 70)],
    "70-100%": missing_pct_sorted[(missing_pct_sorted >= 70) & (missing_pct_sorted <= 100)],
}

# Optional: view each range
for r, s in sorted_missing_ranges.items():
    print(f"\n{r} ({len(s)} features)")
    display(s.to_frame("missing_pct"))

sorted_feature_lists = {k: v.index.tolist() for k, v in sorted_missing_ranges.items()}


0-10% (64 features)


,missing_pct
timestamp,0.00
direction,0.00
target_datarate,0.00
drive_mode,0.00
scenario,0.00
...,...
PCell_Cell_Identity,6.12
PCell_Downlink_frequency,6.12
Pos in Ref Round,8.03
datarate,8.60



10-70% (16 features)


,missing_pct
ping_ms,28.91
PCell_Downlink_RBs_MCS_0,29.19
PCell_Downlink_RBs_MCS_1,41.43
PCell_Downlink_RBs_MCS_31,47.13
PCell_Downlink_RBs_MCS_15,59.22
PCell_Downlink_RBs_MCS_18,59.29
PCell_Downlink_RBs_MCS_14,59.99
PCell_Downlink_RBs_MCS_19,60.77
PCell_Downlink_RBs_MCS_30,61.92
PCell_Downlink_RBs_MCS_13,62.10



70-100% (80 features)


,missing_pct
PCell_Downlink_RBs_MCS_8,73.45
PCell_Downlink_RBs_MCS_9,74.23
PCell_Downlink_RBs_MCS_23,75.16
PCell_Downlink_RBs_MCS_7,76.18
PCell_Downlink_RBs_MCS_3,77.74
...,...
SCell_Downlink_RBs_MCS_17,99.96
SCell_Downlink_RBs_MCS_10,99.97
SCell_Uplink_Num_RBs,100.00
SCell_Uplink_Tx_Power_(dBm),100.00


In [8]:
# Missing % for all features
missing_pct = df.isna().mean().mul(100).round(2)

# Keep only PCell / SCell features
pcell_scell_missing = missing_pct[
    missing_pct.index.str.contains(r"PCell|SCell", case=False, regex=True)
].sort_values(ascending=False)

# Show
display(pcell_scell_missing.to_frame("missing_pct"))

pcell_missing = missing_pct[missing_pct.index.str.contains("PCell", case=False)].sort_values(ascending=False)
scell_missing = missing_pct[missing_pct.index.str.contains("SCell", case=False)].sort_values(ascending=False)

display(pcell_missing.to_frame("missing_pct"))
display(scell_missing.to_frame("missing_pct"))

,missing_pct
SCell_Uplink_Tx_Power_(dBm),100.00
SCell_Uplink_Num_RBs,100.00
SCell_Uplink_TB_Size,100.00
SCell_Downlink_RBs_MCS_10,99.97
SCell_Downlink_RBs_MCS_17,99.96
...,...
PCell_freq_MHz,4.84
PCell_SNR_2,4.84
PCell_E-ARFCN,4.84
PCell_RSRP_2,4.84


,missing_pct
PCell_Downlink_RBs_MCS_10,92.40
PCell_Downlink_RBs_MCS_28,89.02
PCell_Downlink_RBs_MCS_2,87.62
PCell_Downlink_RBs_MCS_17,86.42
PCell_Downlink_RBs_MCS_4,85.71
...,...
PCell_RSRQ_max,4.84
PCell_RSRQ_2,4.84
PCell_RSRQ_1,4.84
PCell_RSRP_max,4.84


,missing_pct
SCell_Uplink_Tx_Power_(dBm),100.00
SCell_Uplink_TB_Size,100.00
SCell_Uplink_Num_RBs,100.00
SCell_Downlink_RBs_MCS_10,99.97
SCell_Downlink_RBs_MCS_17,99.96
...,...
SCell_RSRQ_max,84.27
SCell_RSRQ_2,84.27
SCell_RSRQ_1,84.27
SCell_RSRP_max,84.27


In [9]:
# Get numeric columns
numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()

# Check for constant features
constant_features = []
quasi_constant_features = []


print(f"{'Feature':<50s} {'Constant Values':>15s} {'Most Common %':>15s}")

for col in numeric_cols:
    n_unique = df[col].nunique()
    
    
    if n_unique == 1:
        # Perfectly constant
        constant_features.append(col)
        value = df[col].dropna().iloc[0] if len(df[col].dropna()) > 0 else 'NaN'
        print(f"{col:<50s} {value} {'100.00%':>20s} ")
    
    elif n_unique == 2:
        value_counts = df[col].value_counts()
        most_common = value_counts.index[0]
        most_common_pct = value_counts.iloc[0] / value_counts.sum() * 100
        
        if most_common_pct > 98:
            quasi_constant_features.append(col)
            print(f"{col:<50s} {most_common} {most_common_pct:>14.2f}%")


print(f"    Perfectly constant:  {len(constant_features)} features")
print(f"    constant (>98%): {len(quasi_constant_features)} features")

Feature                                            Constant Values   Most Common %
visibility                                         16.093              100.00% 
PCell_MCC                                          262.0              100.00% 
PCell_MNC_Digit                                    2.0              100.00% 
PCell_Allowed_Access                               0.0          99.93%
SCell_MCC                                          262.0              100.00% 
SCell_MNC_Digit                                    2.0              100.00% 
SCell_Allowed_Access                               0.0          98.56%
    Perfectly constant:  5 features
    constant (>98%): 2 features


In [10]:
constant_features+quasi_constant_features

['visibility',
 'PCell_MCC',
 'PCell_MNC_Digit',
 'SCell_MCC',
 'SCell_MNC_Digit',
 'PCell_Allowed_Access',
 'SCell_Allowed_Access']

In [11]:

# Find completely missing features
completely_missing = []


for col in df.columns:
    missing_pct = df[col].isnull().sum() / len(df) * 100
    if missing_pct == 100:
        completely_missing.append(col)
    
print(f"Completely missing: {len(completely_missing)} features")

if len(completely_missing) > 0:
    print(f"\nFEATURES TO REMOVE:")
    for feat in completely_missing:
        print(f"     {feat}")

features_to_remove_missing = completely_missing



Completely missing: 3 features

FEATURES TO REMOVE:
     SCell_Uplink_Num_RBs
     SCell_Uplink_TB_Size
     SCell_Uplink_Tx_Power_(dBm)


In [12]:
df[features_to_remove_missing].isnull().sum()

SCell_Uplink_Num_RBs           207434
SCell_Uplink_TB_Size           207434
SCell_Uplink_Tx_Power_(dBm)    207434
dtype: int64

In [13]:

numeric_cols = df.select_dtypes(include=[np.number]).columns.tolist()
corr_matrix_full = df[numeric_cols].corr()


perfect_corr_pairs = []

for i in range(len(corr_matrix_full.columns)):
    for j in range(i+1, len(corr_matrix_full.columns)):
        corr_val = corr_matrix_full.iloc[i, j]
        
        if abs(corr_val) >= 0.9999 and not np.isnan(corr_val):  # Nearly perfect
            feat1 = corr_matrix_full.columns[i]
            feat2 = corr_matrix_full.columns[j]
            perfect_corr_pairs.append({
                'Feature_1': feat1,
                'Feature_2': feat2,
                'Correlation': corr_val
            })

perfect_corr_df = pd.DataFrame(perfect_corr_pairs)



print(f" Perfect correlation pairs: {len(perfect_corr_df)}")

if len(perfect_corr_df) > 0:
    print(f"\nPERFECTLY CORRELATED PAIRS:")
    print(f"{'Feature 1':<40s} {'Feature 2':<40s} {'Correlation':>12s}")
    
    for idx, row in perfect_corr_df.iterrows():
        print(f"{row['Feature_1']:<40s} {row['Feature_2']:<40s} {row['Correlation']:>12.6f}")
    
    # Decide which to keep (keep Feature_1, remove Feature_2 by default)
    features_to_remove_redundant = perfect_corr_df['Feature_2'].tolist()
    
    
    print(f"\n(redundant):")
    for feat in features_to_remove_redundant:
        print(f"    {feat}")
else:
    features_to_remove_redundant = []
    print("\n No perfectly correlated features found")



 Perfect correlation pairs: 14

PERFECTLY CORRELATED PAIRS:
Feature 1                                Feature 2                                 Correlation
temperature                              apparentTemperature                          1.000000
PCell_RSSI_2                             PCell_RSSI_max                               1.000000
PCell_E-ARFCN                            PCell_Downlink_frequency                     0.999923
PCell_E-ARFCN                            PCell_Uplink_frequency                       0.999923
PCell_Downlink_frequency                 PCell_Uplink_frequency                       1.000000
PCell_TAC                                PCell_MNC                                    0.999998
SCell_RSSI_2                             SCell_RSSI_max                               1.000000
SCell_Downlink_RBs_MCS_0                 SCell_Downlink_RBs_MCS_17                    1.000000
SCell_Downlink_RBs_MCS_2                 SCell_Downlink_RBs_MCS_17                   

In [14]:
# Identify 1.2%perecntage missing rows

gps_cols = ['Latitude', 'Longitude', 'Altitude', 'speed_kmh', 'COG']
weather_cols = [col for col in df.columns if any(w in col.lower() for w in 
                ['precip', 'temperature', 'dewpoint', 'humidity', 'pressure', 
                 'wind', 'cloud', 'uvindex', 'visibility', 'apparent'])]

gps_all_missing = df[gps_cols].isnull().all(axis=1)
weather_all_missing = df[weather_cols].isnull().all(axis=1)
traffic_missing = df['Traffic Jam Factor'].isnull() if 'Traffic Jam Factor' in df.columns else pd.Series(False, index=df.index)
area_missing = df['area'].isnull() if 'area' in df.columns else pd.Series(False, index=df.index)


all_rows= gps_all_missing & weather_all_missing & traffic_missing & area_missing

In [15]:
print(f"  Total rows with ALL missing: {all_rows.sum():,}")
print(f"  Percentage of dataset: {all_rows.sum()/len(df)*100:.2f}%")

  Total rows with ALL missing: 2,492
  Percentage of dataset: 1.20%


In [16]:
ping_missing_in_bad = df.loc[all_rows, 'ping_ms'].isnull().sum()
datarate_missing_in_bad = df.loc[all_rows, 'datarate'].isnull().sum()

In [17]:
ping_missing_in_bad

np.int64(2256)

In [18]:
datarate_missing_in_bad

np.int64(2159)

In [20]:


print(f"  GPS missing:       {gps_all_missing[all_rows].sum():>6,} / {all_rows.sum():>6,} (100.00%)")
print(f"  Weather missing:   {weather_all_missing[all_rows].sum():>6,} / {all_rows.sum():>6,} (100.00%)")
print(f"  Traffic missing:   {traffic_missing[all_rows].sum():>6,} / {all_rows.sum():>6,} (100.00%)")
print(f"  Area missing:      {area_missing[all_rows].sum():>6,} / {all_rows.sum():>6,} (100.00%)")
print(f"  Ping missing:      {ping_missing_in_bad:>6,} / {all_rows.sum():>6,} ({ping_missing_in_bad/all_rows.sum()*100:>5.2f}%)")
print(f"  Datarate missing:  {datarate_missing_in_bad:>6,} / {all_rows.sum():>6,} ({datarate_missing_in_bad/all_rows.sum()*100:>5.2f}%)")


rows_to_remove = all_rows



  GPS missing:        2,492 /  2,492 (100.00%)
  Weather missing:    2,492 /  2,492 (100.00%)
  Traffic missing:    2,492 /  2,492 (100.00%)
  Area missing:       2,492 /  2,492 (100.00%)
  Ping missing:       2,256 /  2,492 (90.53%)
  Datarate missing:   2,159 /  2,492 (86.64%)


In [22]:
scell_cols_mcs = [col for col in df.columns if col.startswith('SCell_Downlink_RBs_MCS')]
df['scell_downlink_rbs_mcs_mean'] = df[scell_cols_mcs].mean(axis=1)
#df_cleaned['scell_downlink_rbs_mcs_mean'].isnull().sum()/len(df_cleaned)*100

In [25]:
df[scell_cols_mcs]

,SCell_Downlink_RBs_MCS_0,SCell_Downlink_RBs_MCS_1,SCell_Downlink_RBs_MCS_2,SCell_Downlink_RBs_MCS_3,SCell_Downlink_RBs_MCS_4,SCell_Downlink_RBs_MCS_5,SCell_Downlink_RBs_MCS_6,SCell_Downlink_RBs_MCS_7,SCell_Downlink_RBs_MCS_8,SCell_Downlink_RBs_MCS_9,...,SCell_Downlink_RBs_MCS_22,SCell_Downlink_RBs_MCS_23,SCell_Downlink_RBs_MCS_24,SCell_Downlink_RBs_MCS_25,SCell_Downlink_RBs_MCS_26,SCell_Downlink_RBs_MCS_27,SCell_Downlink_RBs_MCS_28,SCell_Downlink_RBs_MCS_29,SCell_Downlink_RBs_MCS_30,SCell_Downlink_RBs_MCS_31
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,668.0,9604.0,32332.0,NaN,NaN,3900.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5417.5,7606.0,9456.5,NaN,NaN,NaN,NaN,NaN,NaN,2406.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,752.0,9652.0,35806.0,NaN,NaN,362.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
207429,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,96.0,888.0,85846.0,NaN,NaN,714.0
207430,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,472.0,8844.0,79734.0,NaN,NaN,564.0
207431,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,892.0,11012.0,62550.0,NaN,NaN,558.0
207432,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,96.0,1300.0,81906.0,NaN,NaN,1338.0


In [26]:
#df_cleaned = df.drop(columns=scell_cols, errors='ignore')

In [27]:
features_to_remove={
    'SCell_Uplink_Num_RBs',
    'SCell_Uplink_TB_Size',
    'SCell_Uplink_Tx_Power_(dBm)',
    'visibility',
 'PCell_MCC',
 'PCell_MNC_Digit',
 'SCell_MCC',
 'SCell_MNC_Digit',
 'PCell_Allowed_Access',
 'SCell_Allowed_Access',
 'apparentTemperature',
 'uvIndex','cloudCover','pressure',
 'dewPoint','PCell_MNC','SCell_MNC','ts_gps'
}



In [29]:

print(f"\n Before removal:")
print(f"     Shape: {df.shape}")
print(f"     Rows: {df.shape[0]:,}")
print(f"     Features: {df.shape[1]}")

# Remove features
df_clean_f = df.drop(columns=features_to_remove, errors='ignore')
df_cleaned = df_clean_f.drop(columns=scell_cols_mcs, errors='ignore')

print(f"\nAfter feature removal:")
print(f"     Shape: {df_cleaned.shape}")
print(f"     Rows: {df_cleaned.shape[0]:,}")
print(f"     Features: {df_cleaned.shape[1]}")
print(f"     Removed: {df.shape[1] - df_cleaned.shape[1]} features")


 Before removal:
     Shape: (207434, 161)
     Rows: 207,434
     Features: 161

After feature removal:
     Shape: (207434, 111)
     Rows: 207,434
     Features: 111
     Removed: 50 features


In [30]:
df_cleaned.head()

,timestamp,device,ping_ms,datarate,jitter,Latitude,Longitude,Altitude,speed_kmh,COG,...,SCell_TAC,SCell_Band_Indicator,SCell_freq_MHz,scenario,drive_mode,target_datarate,direction,measured_qos,operator,scell_downlink_rbs_mcs_mean
0,2021-06-22 09:49:00+02:00,pc4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1494.0,7.0,2600.0,A3D,platoon,350000000,downlink,datarate,1,11626.000000
1,2021-06-22 09:49:00+02:00,pc3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49101.0,3.0,1800.0,A3D,platoon,350000000,downlink,datarate,2,3216.461538
2,2021-06-22 09:49:00+02:00,pc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1494.0,7.0,2600.0,A3D,platoon,350000000,downlink,datarate,1,11643.000000
3,2021-06-22 09:49:00+02:00,pc2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49101.0,7.0,2600.0,A3D,platoon,350000000,downlink,datarate,2,9979.800000
4,2021-06-22 09:49:01+02:00,pc2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49101.0,7.0,2600.0,A3D,platoon,350000000,downlink,datarate,2,15638.333333


#### With High Missing % of SCell Columns we ignore all SCell columns

In [31]:
scell_features = [col for col in df_cleaned.columns if "SCell" in col]
scell_features

['SCell_RSRP_1',
 'SCell_RSRP_2',
 'SCell_RSRP_max',
 'SCell_RSRQ_1',
 'SCell_RSRQ_2',
 'SCell_RSRQ_max',
 'SCell_RSSI_1',
 'SCell_RSSI_2',
 'SCell_RSSI_max',
 'SCell_SNR_1',
 'SCell_SNR_2',
 'SCell_E-ARFCN',
 'SCell_Downlink_Num_RBs',
 'SCell_Downlink_TB_Size',
 'SCell_Downlink_Average_MCS',
 'SCell_Cell_ID',
 'SCell_Downlink_frequency',
 'SCell_Uplink_frequency',
 'SCell_Downlink_bandwidth_MHz',
 'SCell_Uplink_bandwidth_MHz',
 'SCell_Cell_Identity',
 'SCell_TAC',
 'SCell_Band_Indicator',
 'SCell_freq_MHz']

In [32]:
df_cleaned = df_cleaned.drop(columns=scell_features, axis=1, errors="ignore")
df_cleaned.head()

,timestamp,device,ping_ms,datarate,jitter,Latitude,Longitude,Altitude,speed_kmh,COG,...,PCell_TAC,PCell_Band_Indicator,PCell_freq_MHz,scenario,drive_mode,target_datarate,direction,measured_qos,operator,scell_downlink_rbs_mcs_mean
0,2021-06-22 09:49:00+02:00,pc4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1494.0,3.0,1800.0,A3D,platoon,350000000,downlink,datarate,1,11626.000000
1,2021-06-22 09:49:00+02:00,pc3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49101.0,7.0,2600.0,A3D,platoon,350000000,downlink,datarate,2,3216.461538
2,2021-06-22 09:49:00+02:00,pc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1494.0,3.0,1800.0,A3D,platoon,350000000,downlink,datarate,1,11643.000000
3,2021-06-22 09:49:00+02:00,pc2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49101.0,3.0,1800.0,A3D,platoon,350000000,downlink,datarate,2,9979.800000
4,2021-06-22 09:49:01+02:00,pc2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,49101.0,3.0,1800.0,A3D,platoon,350000000,downlink,datarate,2,15638.333333


#### Grouping PCell MCS 32 columns to 3 columms

In [33]:
pcell_features_mcs = [col for col in df_cleaned.columns if "PCell_Downlink_RBs_MCS" in col]
pcell_features_mcs

['PCell_Downlink_RBs_MCS_0',
 'PCell_Downlink_RBs_MCS_1',
 'PCell_Downlink_RBs_MCS_2',
 'PCell_Downlink_RBs_MCS_3',
 'PCell_Downlink_RBs_MCS_4',
 'PCell_Downlink_RBs_MCS_5',
 'PCell_Downlink_RBs_MCS_6',
 'PCell_Downlink_RBs_MCS_7',
 'PCell_Downlink_RBs_MCS_8',
 'PCell_Downlink_RBs_MCS_9',
 'PCell_Downlink_RBs_MCS_10',
 'PCell_Downlink_RBs_MCS_11',
 'PCell_Downlink_RBs_MCS_12',
 'PCell_Downlink_RBs_MCS_13',
 'PCell_Downlink_RBs_MCS_14',
 'PCell_Downlink_RBs_MCS_15',
 'PCell_Downlink_RBs_MCS_16',
 'PCell_Downlink_RBs_MCS_17',
 'PCell_Downlink_RBs_MCS_18',
 'PCell_Downlink_RBs_MCS_19',
 'PCell_Downlink_RBs_MCS_20',
 'PCell_Downlink_RBs_MCS_21',
 'PCell_Downlink_RBs_MCS_22',
 'PCell_Downlink_RBs_MCS_23',
 'PCell_Downlink_RBs_MCS_24',
 'PCell_Downlink_RBs_MCS_25',
 'PCell_Downlink_RBs_MCS_26',
 'PCell_Downlink_RBs_MCS_27',
 'PCell_Downlink_RBs_MCS_28',
 'PCell_Downlink_RBs_MCS_29',
 'PCell_Downlink_RBs_MCS_30',
 'PCell_Downlink_RBs_MCS_31']

In [34]:
# Missing % for all columns in pcell_features_mcs
pcell_mcs_missing_pct = (
    df_cleaned[pcell_features_mcs]
    .isna()
    .mean()
    .mul(100)
    .round(2)
    .sort_values(ascending=True)
)

# Display as table
display(
    pcell_mcs_missing_pct
    .rename("missing_pct")
    .reset_index()
    .rename(columns={"index": "feature"})
)

,feature,missing_pct
0,PCell_Downlink_RBs_MCS_0,29.19
1,PCell_Downlink_RBs_MCS_1,41.43
2,PCell_Downlink_RBs_MCS_31,47.13
3,PCell_Downlink_RBs_MCS_15,59.22
4,PCell_Downlink_RBs_MCS_18,59.29
5,PCell_Downlink_RBs_MCS_14,59.99
6,PCell_Downlink_RBs_MCS_19,60.77
7,PCell_Downlink_RBs_MCS_30,61.92
8,PCell_Downlink_RBs_MCS_13,62.10
9,PCell_Downlink_RBs_MCS_21,62.38


In [35]:
import re

def mcs_idx(col):
    m = re.search(r"_(\d+)$", col)
    return int(m.group(1)) if m else None

low_cols  = [c for c in pcell_features_mcs if mcs_idx(c) is not None and 0  <= mcs_idx(c) <= 6]
mid_cols  = [c for c in pcell_features_mcs if mcs_idx(c) is not None and 7  <= mcs_idx(c) <= 16]
high_cols = [c for c in pcell_features_mcs if mcs_idx(c) is not None and 17 <= mcs_idx(c) <= 31]

# Replace missing values with 0 before aggregation
df_cleaned["PCell_DL_RBs_MCS_Low"] = df_cleaned[low_cols].sum(axis=1)
df_cleaned["PCell_DL_RBs_MCS_Mid"] = df_cleaned[mid_cols].sum(axis=1)
df_cleaned["PCell_DL_RBs_MCS_High"] = df_cleaned[high_cols].sum(axis=1)

In [36]:
df_cleaned[low_cols]

,PCell_Downlink_RBs_MCS_0,PCell_Downlink_RBs_MCS_1,PCell_Downlink_RBs_MCS_2,PCell_Downlink_RBs_MCS_3,PCell_Downlink_RBs_MCS_4,PCell_Downlink_RBs_MCS_5,PCell_Downlink_RBs_MCS_6
0,NaN,156.0,NaN,5.0,NaN,NaN,NaN
1,NaN,108.0,NaN,NaN,NaN,NaN,NaN
2,3.0,96.0,NaN,NaN,NaN,NaN,NaN
3,8.0,92.0,NaN,NaN,NaN,NaN,NaN
4,4.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
207429,3.0,60.0,NaN,NaN,NaN,NaN,NaN
207430,3.0,4.0,NaN,NaN,NaN,NaN,NaN
207431,NaN,80.0,NaN,5.0,NaN,NaN,NaN
207432,NaN,180.0,NaN,5.0,NaN,NaN,NaN


In [37]:
df_cleaned = df_cleaned.drop(columns=pcell_features_mcs, axis=1, errors="ignore")
df_cleaned.head()

,timestamp,device,ping_ms,datarate,jitter,Latitude,Longitude,Altitude,speed_kmh,COG,...,scenario,drive_mode,target_datarate,direction,measured_qos,operator,scell_downlink_rbs_mcs_mean,PCell_DL_RBs_MCS_Low,PCell_DL_RBs_MCS_Mid,PCell_DL_RBs_MCS_High
0,2021-06-22 09:49:00+02:00,pc4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A3D,platoon,350000000,downlink,datarate,1,11626.000000,161.0,0.0,39709.0
1,2021-06-22 09:49:00+02:00,pc3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A3D,platoon,350000000,downlink,datarate,2,3216.461538,108.0,35212.0,10232.0
2,2021-06-22 09:49:00+02:00,pc1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A3D,platoon,350000000,downlink,datarate,1,11643.000000,99.0,0.0,36711.0
3,2021-06-22 09:49:00+02:00,pc2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A3D,platoon,350000000,downlink,datarate,2,9979.800000,100.0,931.0,40280.0
4,2021-06-22 09:49:01+02:00,pc2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,A3D,platoon,350000000,downlink,datarate,2,15638.333333,4.0,718.0,26547.0


In [38]:
# Missing percentage for each feature in df_engineered
feature_missing_pct = (
    df_cleaned.isna()
    .mean()
    .mul(100)
    .round(2)
    .sort_values(ascending=False)
)

# Convert to a feature list with percentages
feature_missing_list = [
    {"feature": col, "missing_pct": pct}
    for col, pct in feature_missing_pct.items()
]

# Optional: DataFrame view
feature_missing_df = pd.DataFrame(feature_missing_list)

feature_missing_df

,feature,missing_pct
0,scell_downlink_rbs_mcs_mean,85.85
1,ping_ms,28.91
2,datarate,8.60
3,jitter,8.60
4,Pos in Ref Round,8.03
5,PCell_Downlink_frequency,6.12
6,PCell_Uplink_bandwidth_MHz,6.12
7,PCell_Cell_ID,6.12
8,PCell_Uplink_frequency,6.12
9,PCell_Downlink_bandwidth_MHz,6.12


In [39]:
# Verify all features removed
actually_removed = set(df.columns) - set(df_cleaned.columns)
print(f"\n removed: {len(actually_removed)} features")



 removed: 106 features


In [40]:
print(f"  Total rows with ALL missing: {all_rows.sum():,}")
print(f"  Percentage of dataset: {all_rows.sum()/len(df)*100:.2f}%")

  Total rows with ALL missing: 2,492
  Percentage of dataset: 1.20%


In [41]:
# Remove 2492 rows
df_final = df_cleaned[~all_rows].copy()

In [42]:
for col in df_final:
    if col.startswith('SCell_ID'):
        col.value_count()

In [43]:
print(f"\n After row removal:")
print(f"     Shape: {df_final.shape}")
print(f"     Rows: {df_final.shape[0]:,}")
print(f"     Removed: {all_rows.sum():,} rows ({all_rows.sum()/len(df_cleaned)*100:.2f}%)")
print(f"     Retention: {df_final.shape[0]/df.shape[0]*100:.2f}%")


 After row removal:
     Shape: (204942, 58)
     Rows: 204,942
     Removed: 2,492 rows (1.20%)
     Retention: 98.80%


In [44]:
print(f"\n TRANSFORMATION:")
print(f"   Original:  {df.shape[0]:>7,} rows × {df.shape[1]:>3d} features")
print(f"   Final:     {df_final.shape[0]:>7,} rows × {df_final.shape[1]:>3d} features")
print(f"   ")
print(f"   Rows:      -{df.shape[0] - df_final.shape[0]:>6,} ({-(df.shape[0] - df_final.shape[0])/df.shape[0]*100:>5.2f}%)")
print(f"   Features:  -{df.shape[1] - df_final.shape[1]:>6,}")

# Calculate missingness
total_cells_original = df.shape[0] * df.shape[1]
missing_cells_original = df.isnull().sum().sum()

total_cells_final = df_final.shape[0] * df_final.shape[1]
missing_cells_final = df_final.isnull().sum().sum()

print(f"\n MISSINGNESS:")
print(f"   Original:  {missing_cells_original:>10,} / {total_cells_original:>10,} ({missing_cells_original/total_cells_original*100:>5.2f}%)")
print(f"   Final:     {missing_cells_final:>10,} / {total_cells_final:>10,} ({missing_cells_final/total_cells_final*100:>5.2f}%)")
print(f"   Change:    {missing_cells_final/total_cells_final*100 - missing_cells_original/total_cells_original*100:>+5.2f} percentage points")





 TRANSFORMATION:
   Original:  207,434 rows × 161 features
   Final:     204,942 rows ×  58 features
   
   Rows:      - 2,492 (-1.20%)
   Features:  -   103

 MISSINGNESS:
   Original:  17,446,982 / 33,396,874 (52.24%)
   Final:        581,394 / 11,886,636 ( 4.89%)
   Change:    -47.35 percentage points


In [45]:
# Feature breakdown
numeric_cols_final = df_final.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols_final = df_final.select_dtypes(include=['object', 'category']).columns.tolist()
datetime_cols_final = df_final.select_dtypes(include=['datetime64']).columns.tolist()

print(f"\nFEATURE TYPES:")
print(f"    Numeric:      {len(numeric_cols_final):>3d}")
print(f"    Categorical:  {len(categorical_cols_final):>3d}")
print(f"    Datetime:     {len(datetime_cols_final):>3d}")


FEATURE TYPES:
    Numeric:       48
    Categorical:    9
    Datetime:       0


In [46]:


# Save cleaned dataset
df_final.to_pickle('results/df_final.pkl')



df_final.to_parquet('results/df_final.parquet')

# Save removal summary
removal_summary = {
    'timestamp': pd.Timestamp.now().isoformat(),
    'original_shape': list(df.shape),
    'final_shape': list(df_final.shape),
    'rows_removed': int(df.shape[0] - df_final.shape[0]),
    'features_removed': int(df.shape[1] - df_final.shape[1]),
    'removed_features': list(features_to_remove),
    'removal_breakdown': {
        'completely_missing': ['SCell_Uplink_Num_RBs', 'SCell_Uplink_TB_Size', 'SCell_Uplink_Tx_Power_(dBm)'],
        'quasi_constant': ['visibility', 'PCell_MCC', 'PCell_MNC_Digit', 'SCell_MCC', 'SCell_MNC_Digit', 
                          'PCell_Allowed_Access', 'SCell_Allowed_Access'],
        'redundant': ['apparentTemperature']
    },
    'bad_rows_removed': int(all_rows.sum()),
    'retention_rate': float(df_final.shape[0] / df.shape[0]),
    'final_missing_pct': float(missing_cells_final / total_cells_final * 100)
}

import json
with open('results/cleaning_summary.json', 'w') as f:
    json.dump(removal_summary, f, indent=2)


# Save feature list
remaining_features = pd.DataFrame({
    'feature': df_final.columns,
    'dtype': df_final.dtypes.astype(str).values,
    'missing_count': df_final.isnull().sum().values,
    'missing_pct': (df_final.isnull().sum() / len(df_final) * 100).values,
    'unique_values': [df_final[col].nunique() for col in df_final.columns]
})
remaining_features = remaining_features.sort_values('missing_pct', ascending=False)
remaining_features.to_csv('results/cleaned_features_list.csv', index=False)
print("Saved: results/cleaned_features_list_1.csv")




Saved: results/cleaned_features_list_1.csv
